In [1]:
# import bus_processing
import bus_prediction
import os
import pandas as pd
from constants import DATA_DIR

In [2]:
def load_bus_data():
    path = os.path.join(DATA_DIR, "dataset.csv")
    df = pd.read_csv(path)
    return df

bus_data = load_bus_data()
bus_data["time_bucket"] = pd.to_datetime(bus_data["time_bucket"], utc=True)
X_train, y_train, X_test, y_test = bus_prediction.prepare_bus_data(bus_data)

In [3]:
lstm_model = bus_prediction.LSTMModule(input_size=X_train.shape[1], hidden_size=64, num_layers=5, dropout=0.1)
lstm_predictor = bus_prediction.LSTMPredictor(lstm_model, window_size=10)

In [4]:
lstm_predictor.train(X_train.values, y_train.values, epochs=20)

/Users/legendword/Documents/zwh/Coding/CPSC440/cpsc440-transit-project/src/bus_prediction.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  self.X = torch.tensor(X_lagged, dtype=torch.float32)


Epoch 1/20, Train Loss: 1.5525, Val Loss: 0.9382
Epoch 2/20, Train Loss: 1.4630, Val Loss: 0.9361
Epoch 3/20, Train Loss: 1.4425, Val Loss: 0.9214
Epoch 4/20, Train Loss: 1.4374, Val Loss: 0.9739
Epoch 5/20, Train Loss: 1.4280, Val Loss: 0.9061
Epoch 6/20, Train Loss: 1.3419, Val Loss: 0.6854
Epoch 7/20, Train Loss: 1.2687, Val Loss: 0.7564
Epoch 8/20, Train Loss: 1.2263, Val Loss: 0.8895
Epoch 9/20, Train Loss: 1.2565, Val Loss: 0.8082
Epoch 10/20, Train Loss: 1.2202, Val Loss: 0.7821
Epoch 11/20, Train Loss: 1.1493, Val Loss: 0.8252
Epoch 12/20, Train Loss: 1.1171, Val Loss: 0.7651
Epoch 13/20, Train Loss: 1.1002, Val Loss: 0.7200
Epoch 14/20, Train Loss: 1.0370, Val Loss: 0.5758
Epoch 15/20, Train Loss: 0.8922, Val Loss: 0.4854
Epoch 16/20, Train Loss: 0.8272, Val Loss: 0.6562
Epoch 17/20, Train Loss: 0.7443, Val Loss: 0.5994
Epoch 18/20, Train Loss: 0.6803, Val Loss: 0.4755
Epoch 19/20, Train Loss: 0.6597, Val Loss: 0.5347
Epoch 20/20, Train Loss: 0.6150, Val Loss: 0.4041


In [5]:
mse = lstm_predictor.evaluate(X_test.values, y_test.values)
print(f"Test MSE: {mse:.4f}")

Shape of X_lagged: (2192, 10, 23)
Shape of y_pred: (2192,)
Shape of test_y: (2192,)
Test MSE: 1.2167


In [6]:
xgboost_predictor = bus_prediction.XGBoostPredictor()
xgboost_predictor.train(X_train.values, y_train.values)
xgboost_mse = xgboost_predictor.evaluate(X_test.values, y_test.values)
print(f"XGBoost Test MSE: {xgboost_mse:.4f}")

Validation MSE: 0.0107
Shape of y_pred: (2202,)
Shape of test_y: (2202,)
XGBoost Test MSE: 0.0689
